In [5]:
import pandas as pd

train_df = pd.read_csv("./data/train.csv")
for column in train_df.columns:
    if train_df[column].dtype=='object':
        train_df[column],unique=pd.factorize(train_df[column])
corelation=train_df.corrwith(train_df['SalePrice']).sort_values(ascending=False)
corelation.head(50)


SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
FireplaceQu      0.402854
BsmtFinSF1       0.386420
MasVnrType       0.379400
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotShape         0.267759
LotArea          0.263843
GarageFinish     0.247470
BsmtExposure     0.244946
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
RoofStyle        0.159332
SaleCondition    0.142503
ScreenPorch      0.111447
LotConfig        0.109106
PoolArea         0.092404
LandContour      0.092009
GarageQual       0.083479
SaleType         0.072896
BsmtQual         0.058965
PoolQC      